# Profile NeRF with Timeloop and Accelergy

In [1]:
# Add parent dir to path so we can import accelerating_nerfs
import sys
sys.path.append("../")

In [2]:
import json
import numpy as np
import os
import pandas as pd
import re
import yaml
import traceback

from collections import defaultdict
from accelerating_nerfs.models import VanillaNeRF, patch_forward

# Custom code
from analysis import *
from profiler import Profiler
from notebook_utils import natural_sort

## Load NeRF model
We use vanilla NeRFs which are MLPs. Uncomment the cell below to view the architecture.

In [3]:
# Uncomment to view architecture diagram
# from IPython.display import IFrame
# IFrame("./figures/netdiag-modified.pdf", width=600, height=325)

In [4]:
model = VanillaNeRF()

# We need to patch the forward method for the purpose of mapping to pass in ray directions
# This ensures the bottleneck layer is captured by the converter from pytorch2timeloop 
patch_forward(model)
print(model)

VanillaNeRF(
  (posi_encoder): SinusoidalEncoder()
  (view_encoder): SinusoidalEncoder()
  (mlp): NerfMLP(
    (base): MLP(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList(
        (0): Linear(in_features=63, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Linear(in_features=319, out_features=256, bias=True)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (sigma_layer): DenseLayer(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList()
      (output_layer): Linear(in_features=256, out_features=1, bias=True)
    )
    (bottl

/home/workspace/notebooks/../accelerating_nerfs/models.py:254: UserWarning: patched forward of VanillaNeRF to also pass the condition. You should only use this for debugging or with Timeloop and Accelergy
  warnings.warn(


## Convert to Timeloop and load sparsity into the layers

In [5]:
# TODO: play around with the batch size
batch_size = 128

# The unknown module type warnings are ok
# We create a copy for nerf-sparse so we can copy the layer sparsities over to the configurations
_ = convert_nerf_to_timeloop(model, batch_size=batch_size, sub_dir="nerf")
_ = convert_nerf_to_timeloop(model, batch_size=batch_size, sub_dir="nerf-sparse")
nerf_layer_shapes = load_nerf_layer_shapes()

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unk

Converted VanillaNeRF model to Timeloop problems in workloads/nerf
Converted VanillaNeRF model to Timeloop problems in workloads/nerf-sparse


In [6]:
!ls ../accelerating_nerfs/sparsity/

2023-05-03_00-00-30_sparsity.json  2023-05-03_18-13-41_volrend_sparsity.json
2023-05-03_00-21-28_sparsity.json


In [7]:
# Load layer sparsity results
sparsities = load_nerf_sparsities("../accelerating_nerfs/sparsity/2023-05-03_00-21-28_sparsity.json")
layer_to_avg_sparsity = compute_layer_sparsities(sparsities)
print("Layer to Average Sparsity:", json.dumps(layer_to_avg_sparsity, indent=4))

Loaded sparsity results for dict_keys(['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship'])
Layer to Average Sparsity: {
    "1": 4.346944171693548e-08,
    "2": 0.5232157788498581,
    "3": 0.6606506746276803,
    "4": 0.6719614964550145,
    "5": 0.635327070883858,
    "6": 0.49655377627334907,
    "7": 0.5577130213002062,
    "8": 0.6088893305723659,
    "9": 0.5989934303659363,
    "10": 0.5785213852133457,
    "11": 2.1946953467908288e-09,
    "12": 0.6979629018407192
}


### Load the sparsity results into the Timeloop layers

In [8]:
add_sparsity_to_nerf_layers(layer_to_avg_sparsity, layer_dir="workloads/nerf-sparse")

Layer 1 added densities: {'Inputs': 0.9999999565305583, 'Weights': 1.0, 'Outputs': 0.9999999565305583}
Layer 2 added densities: {'Inputs': 0.47678422115014185, 'Weights': 1.0, 'Outputs': 0.47678422115014185}
Layer 3 added densities: {'Inputs': 0.33934932537231965, 'Weights': 1.0, 'Outputs': 0.33934932537231965}
Layer 4 added densities: {'Inputs': 0.32803850354498554, 'Weights': 1.0, 'Outputs': 0.32803850354498554}
Layer 5 added densities: {'Inputs': 0.364672929116142, 'Weights': 1.0, 'Outputs': 0.364672929116142}
Layer 6 added densities: {'Inputs': 0.5034462237266509, 'Weights': 1.0, 'Outputs': 0.5034462237266509}
Layer 7 added densities: {'Inputs': 0.44228697869979383, 'Weights': 1.0, 'Outputs': 0.44228697869979383}
Layer 8 added densities: {'Inputs': 0.39111066942763406, 'Weights': 1.0, 'Outputs': 0.39111066942763406}
Layer 9 added densities: {'Inputs': 0.40100656963406367, 'Weights': 1.0, 'Outputs': 0.40100656963406367}
Layer 10 added densities: {'Inputs': 0.42147861478665427, 'Weig

### Configure saving of profiling results
This isn't important so you can ignore the details.

In [9]:
# Accumulate results in this dictionary
profile_results = {}

# Setup saving the profiling results
results_dir = "profile_results"
os.makedirs(results_dir, exist_ok=True)


def save_results():
    all_other_results = {}
    
    for arch, arch_results in profile_results.items():
        # Write the super long results to it's own file
        arch_results_path = os.path.join(results_dir, f"{arch}_results.json")
        with open(arch_results_path, "w") as f:
            json.dump(arch_results["results"], f, indent=4)
        
        # Accumulate the other results as they're shorter and more readable
        other_results = {
            k: v for k, v in arch_results.items()
            if k != "results"
        }
        # Have a pointer to the separate results file
        other_results["results"] = os.path.abspath(arch_results_path)
        all_other_results[arch] = other_results
    
    results_path = os.path.join(results_dir, "results.json")
    with open(results_path, "w") as f:
        json.dump(all_other_results, f, indent=4)

    print(f"Saved profile results to {results_path}")

## Profile using Timeloop and Accelergy
I think we can safely ignore the 'No handlers found'

To rerun things, delete the existing results in the `profiled_libs/` directory.

In [ ]:
# Don't use simba_like or simple_output_stationary as the mapper constraints are too stringent
archs_and_sparse = [
    ("eyeriss_like", False),
    ("simple_weight_stationary", False),
    ("eyeriss_like_onchip_compression", True)
]
failed_archs = set()

for (arch, is_sparse) in archs_and_sparse:
    msg = f"Running {arch}"
    print(len(msg) * '=')
    print(msg)
    print(len(msg) * '=')
    
    # Profile - you shouldn't need to change anything here
    try:
        profiler = Profiler(
            top_dir='workloads',
            sub_dir='nerf' if not is_sparse else 'nerf-sparse',
            timeloop_dir=f"designs/{arch}",
            arch_name=arch,
            model=model,
            input_size=(1, 3),
        )
        results, summary, layer_summary = profiler.profile()
    except Exception as e:
        traceback.print_exc()
        print(f"ERROR: could not run profiler for {arch}, do not trust these results!")
        failed_archs.add(arch)
        continue
    
    # Add nerf layer shapes to the layer summary
    for layer_id in layer_summary:
        layer_summary[layer_id].update(nerf_layer_shapes[layer_id])
        
    # Print summary information
    for k, v in summary.items():
        print(f"{k}: {v}")
        
    profile_results[arch] = {
        "results": results,
        "summary": summary,
        "layer_summary": layer_summary,
    }
    save_results()

Running eyeriss_like


running timeloop to get energy and latency...: 100%|██████████| 6/6 [01:05<00:00, 10.93s/it]
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::sin". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


Saved profiled lib to ./profiled_libs/eyeriss_like_profiled_lib.json
total_area: 0.0
total_energy: 1211.45
total_cycle: 1136384.0
num_params: 595844
macs: 593450
activation_size: 2300.0
Saved profile results to profile_results/results.json
Running simple_weight_stationary


running timeloop to get energy and latency...: 100%|██████████| 6/6 [03:39<00:00, 36.50s/it]
Traceback (most recent call last):
  File "/tmp/ipykernel_796/3037256322.py", line 25, in <module>
    results, summary, layer_summary = profiler.profile()
  File "/home/workspace/notebooks/profiler.py", line 348, in profile
    self.populate_profiled_lib(layer_info)
  File "/home/workspace/notebooks/profiler.py", line 297, in populate_profiled_lib
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
  File "/home/workspace/notebooks/profiler.py", line 297, in <dictcomp>
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
KeyError: 'energy'


CRITICAL WARNING: /home/workspace/notebooks/designs/simple_weight_stationary/nerf/layer1/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simple_weight_stationary/nerf/layer6/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simple_weight_stationary/nerf/layer11/timeloop-mapper.stats.txt does not exist, skipping...
ERROR: could not run profiler for simple_weight_stationary, do not trust these results!
Running eyeriss_like_onchip_compression
Sparse optimization is enabled for layer 1
Sparse optimization is enabled for layer 2
Sparse optimization is enabled for layer 3
Sparse optimization is enabled for layer 4
Sparse optimization is enabled for layer 5
Sparse optimization is enabled for layer 6
Sparse optimization is enabled for layer 7
Sparse optimization is enabled for layer 8
Sparse optimization is enabled for layer 9
Sparse optimization is enabled for layer 10
Sparse opti

running timeloop to get energy and latency...:  33%|███▎      | 4/12 [00:37<01:19,  9.98s/it]

## Analyze the results

In [ ]:
# Massage results into dataframes
layer_dfs = {}
all_summary = {}

for arch, arch_results in profile_results.items():
    all_summary[arch] = arch_results["summary"]

    # Load layer results into dataframe
    df_layer = pd.DataFrame.from_dict(arch_results["layer_summary"], orient="index")
    df_layer = df_layer.drop('name', axis=1)
    df_layer.index.name = "layer_id"
    layer_dfs[arch] = df_layer
    
df_summary = pd.DataFrame.from_dict(all_summary, orient="index")

### Overall Summary

In [ ]:
df_summary

In [ ]:
layer_dfs["eyeriss_like"]

In [ ]:
layer_dfs["simple_weight_stationary"]

In [ ]:
layer_dfs["eyeriss_like_onchip_compression"]